# సింపుల్ ఇమేజ్ క్లాసిఫైయర్

ఈ నోట్‌బుక్ మీరు ప్రీ-ట్రెయిన్డ్ న్యూరల్ నెట్‌వర్క్ ఉపయోగించి చిత్రాలను ఎలా వర్గీకరించాలో చూపిస్తుంది.

**మీరు నేర్చుకునేది:**
- ప్రీ-ట్రెయిన్డ్ మోడల్‌ను ఎలా లోడ్ చేసి ఉపయోగించాలి
- చిత్రం ప్రీప్రాసెసింగ్
- చిత్రాలపై అంచనాలు చేయడం
- విశ్వాస స్కోర్లను అర్థం చేసుకోవడం

**వినియోగం:** చిత్రాల్లోని వస్తువులను గుర్తించడం (ఉదా: "పిల్లి", "నక్క", "కారు" మొదలైనవి)

---


## దశ 1: అవసరమైన లైబ్రరీలను దిగుమతి చేసుకోండి

మనం అవసరమైన టూల్స్‌ను దిగుమతి చేసుకుందాం. మీరు ఇవన్నీ ఇప్పటికీ పూర్తిగా అర్థం చేసుకోకపోయినా చింతించకండి!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## దశ 2: ముందుగా శిక్షణ పొందిన మోడల్‌ను లోడ్ చేయండి

మనం **MobileNetV2** ను ఉపయోగించబోతున్నాము, ఇది మిలియన్ల చిత్రాలపై ఇప్పటికే శిక్షణ పొందిన న్యూరల్ నెట్‌వర్క్.

ఇది **ట్రాన్స్‌ఫర్ లెర్నింగ్** అని పిలవబడుతుంది - ఎవరో మరొకరు శిక్షణ ఇచ్చిన మోడల్‌ను ఉపయోగించడం!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## దశ 3: సహాయక ఫంక్షన్లు

మన మోడల్ కోసం చిత్రాలను లోడ్ చేసి సిద్ధం చేయడానికి ఫంక్షన్లు తయారు చేద్దాం.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## దశ 4: నమూనా చిత్రాలపై పరీక్షించండి

ఇంటర్నెట్ నుండి కొన్ని చిత్రాలను వర్గీకరించడానికి ప్రయత్నిద్దాం!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### ప్రతి చిత్రం వర్గీకరించండి


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## దశ 5: మీ స్వంత చిత్రాలను ప్రయత్నించండి!

క్లాసిఫై చేయదలచుకున్న ఏదైనా చిత్రం URL ని క్రింద ఇచ్చిన URL తో మార్చండి.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 ఏమి జరిగింది?

1. **మేము ఒక ప్రీ-ట్రెయిన్డ్ మోడల్‌ను లోడ్ చేసుకున్నాము** - MobileNetV2 మిలియన్ల చిత్రాలపై శిక్షణ పొందింది  
2. **మేము చిత్రాలను ప్రీప్రాసెస్ చేసుకున్నాము** - మోడల్ కోసం వాటిని రీసైజ్ చేసి ఫార్మాట్ చేసాము  
3. **మోడల్ అంచనాలు చేసింది** - 1000 వస్తువుల తరగతుల కోసం సంభావ్యతలను ఇచ్చింది  
4. **మేము ఫలితాలను డీకోడ్ చేసుకున్నాము** - సంఖ్యలను మనుషులకి అర్థమయ్యే లేబుల్స్‌గా మార్చాము  

### విశ్వాస స్కోర్లను అర్థం చేసుకోవడం

- **90-100%**: చాలా విశ్వాసంగా (దాదాపు ఖచ్చితంగా సరైనది)  
- **70-90%**: విశ్వాసంగా (సాధ్యమైనంత వరకు సరైనది)  
- **50-70%**: కొంతమేర విశ్వాసంగా (సరైనదే కావచ్చు)  
- **50% కంటే తక్కువ**: అంతగా విశ్వాసం లేదు (అనిశ్చితి)  

### అంచనాలు తప్పు కావడానికి కారణాలు ఏమిటి?

- **అసాధారణ కోణం లేదా లైటింగ్** - మోడల్ సాధారణ ఫోటోలపై శిక్షణ పొందింది  
- **బహుళ వస్తువులు** - మోడల్ ఒక ప్రధాన వస్తువును ఆశిస్తుంది  
- **దుర్లభ వస్తువులు** - మోడల్ కేవలం 1000 వర్గాలను మాత్రమే తెలుసుకుంటుంది  
- **తక్కువ నాణ్యత గల చిత్రం** - మబ్బుగా లేదా పిక్సెల్ అయిపోయిన చిత్రాలు కష్టంగా ఉంటాయి  

---


## 🚀 తదుపరి దశలు

1. **వివిధ చిత్రాలను ప్రయత్నించండి:**
   - [Unsplash](https://unsplash.com) లో చిత్రాలను వెతకండి
   - రైట్-క్లిక్ → "Copy image address" ద్వారా URL పొందండి

2. **ప్రయోగాలు చేయండి:**
   - అభిజ్ఞాత కళతో ఏమవుతుంది?
   - ఇది వేర్వేరు కోణాల నుండి వస్తువులను గుర్తించగలదా?
   - ఇది బహుళ వస్తువులను ఎలా నిర్వహిస్తుంది?

3. **మరింత తెలుసుకోండి:**
   - [కంప్యూటర్ విజన్ పాఠాలు](../lessons/4-ComputerVision/README.md) అన్వేషించండి
   - మీ స్వంత చిత్రం వర్గీకర్తను శిక్షణ ఇవ్వడం నేర్చుకోండి
   - CNNs (Convolutional Neural Networks) ఎలా పనిచేస్తాయో అర్థం చేసుకోండి

---

## 🎉 అభినందనలు!

మీరు ఒక ఆధునిక న్యూరల్ నెట్‌వర్క్ ఉపయోగించి చిత్రం వర్గీకర్తను సృష్టించారు!

ఈ అదే సాంకేతికతను ఉపయోగిస్తారు:
- Google Photos (మీ ఫోటోలను క్రమబద్ధీకరించడం)
- స్వయం నడిచే కార్లు (వస్తువులను గుర్తించడం)
- వైద్య నిర్ధారణ (X-రేలను విశ్లేషించడం)
- నాణ్యత నియంత్రణ (లోపాలను గుర్తించడం)

కొనసాగిస్తూ అన్వేషించండి మరియు నేర్చుకోండి! 🚀


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారుల బాధ్యత మేము తీసుకోము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
